In [7]:
#zadávání jednotlivých proměnných
#vytvoření časové značky, kde se zobrazí aktuální datum a čas 
from datetime import datetime
timestamp = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
# vytvoření univerzální značky 
import uuid 
id = uuid.uuid1()
#vytvoření proměnné pro mapu, která bude screenshot z mapového pole z připojené mapové komponenty a propsán jako fotka do té proměnné
    # tady bude proměnná mapa jako screenshot z mapové komponenty, kde proběhl výběr území 
    
    # proměnná výběru tématických vrstev 

    # proměnná hodnoty

    # proměnná opatření 

    # proměnná předpokládaného efektu opatření případně co by hrozilo kdyby se opatření nepřijala



In [19]:
# vytvoření html šablony, kde se budou propisovat jednotlivé proměnné
html = """
<!DOCTYPE html>
<html>
<head>
    <title>Zranitelnost krajiny</title>
			<meta charset='UTF-8'>
			<meta name='description' content='HTML stránka georeportu zranitelnosti krajiny'>
			<meta name='keywords' content='zranitelnost krajiny, georeport, ekologie'>
			<meta name='author' content='Adéla Petřeková'>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
        }}
        h1 {{
            background-color: #333;
            color: #fff;
            padding: 10px;
        }}
        p {{
            padding: 10px;
        }}
    </style>

</head>
<body>
   <header>
    <h1>Název reportu: Zranitelnost krajiny</h1>
    <p>Lokalita: XXX (GPS souřadnice) <br>
    Souřadnice: MBR extentu <br>
    Datum a čas generování: {timestamp} <br>
    unikatní identifikátor: {id} asi
    </p>
    </header>
    <main>
    <p> Tento report podává zprávu o degradaci krajiny na základě metody ESAI ....</p>

    <h2> Krok I: Náchylnost krajiny k degradaci </h2>
        <p> Ve zvoleném území dosahuje výsledná hodnota náchylnosti k degradaci hodnoty "XX" a je klasifikována jako "textový název kategorie (číslo kategorie)"</p>
        <table>
            
        </table>
        <--! do ráměčku vložení 3 nejhorších indikátorů -->
        <--! vložení sloupcového grafu -->
        <h3> Věrohodnost dat </h3>
            <table>
                <tr>
                    <th> Zkratka </th>
                    <th> Název </th>
                    <th> Datový zdroj </th>
                    <th> Garant </th>
                    <th> Měřítko </th>
                    <th> Rok aktualizace </th>
                </tr>
                <tr>
                    <td> Indikátor 1 </td>
                    <td> Hodnota 1 </td>
                    <td> Věrohodnost 1 </td>
                </tr>   
            </table>
            <--! mapa území s celkovou kategorií -->

    <h2> Krok II: Posouzení funkčnosti krajiny </h2>
        <p> slouží k nalezení krajinných segmentů jejichž degradace bude mít velký/malý význam dopad na funkčnost krajiny </p>
        <table>
            <tr>
                <th> </th>
                <th> Hodnota </th>
                <th> Kategorie </th>
            </tr>
            <tr>
                <td> Zásoba uhlíku ve vegetaci </td>
                <td> Hodnota 1 </td>
                <td> Kategorie 1 </td>
            </tr>
            <tr>
                <td> Evapotranspirace </td>
                <td> Hodnota 1 </td>
                <td> Kategorie 1 </td>
            </tr>
            <tr>
                <td> Ekologická hodnota biotopu </td>
                <td> Hodnota 1 </td>
                <td> Kategorie 1 </td>
            </tr>
            <tr>
                <td> Vodoretence </td>
                <td> Hodnota 1 </td>
                <td> Kategorie 1 </td>
            </tr>
            <tr>
                <td> Integrovaná hodnota </td>
                <td> --- </td>
                <td> Kategorie 1 </td>
            </tr>
        </table>
        <--! vložení map - 5x  -->
            <h3> Věrohodnost dat </h3>
            <table>
                <tr>
                    <th> Název </th>
                    <th> Datový zdroj geometrie/KB</th>
                    <th> Garant </th>
                    <th> Měřítko </th>
                    <th> Rok aktualizace </th>
                </tr>
                <tr>
                    <td> Indikátor 1 </td>
                    <td> Hodnota 1 </td>
                    <td> Věrohodnost 1 </td>
                </tr>
            </table>

    <h2> Krok III: Odolnost krajiny </h2>
        <--! vložení mapy odolnosti -->

    <h2> Krok IV: Syntéze/lokalita prioritních opatření </h2>
        <--! vložení mapy kde prioritně umístit opatření -->    
        
   
    </main>
    <footer>
    <--! vložení vodorovné čárky, čísla stránky, logo KGI a UVGZ -->

    </footer>


</body>
</html>
"""
#css nedavat moc jako samostatny soubor ale dat to do primo te sablony

In [21]:
#vytvoření HTML souboru ze šablony
with open("index.html", "w") as f:
    f.write(html.format(timestamp=timestamp, id=id))
#otevření html souboru
#print("Generated index.html")
#import webbrowser
#webbrowser.open("index.html")



In [22]:
# output html as a PDF
# first, import the necessary modules

from weasyprint import HTML

In [23]:
# then, convert the HTML to a PDF
HTML("index.html").write_pdf("index.pdf")

print("Generated index.pdf")
#otevření pdf souboru
import webbrowser
webbrowser.open("index.pdf")

Generated index.pdf


True

# api esri na vytvoření bodu, linie a polygonu
https://developers.arcgis.com/python/guide/add-a-point-line-and-polygon/ 
# tutorial esri
 https://developers.arcgis.com/documentation/mapping-apis-and-services/tutorials/

 co následně bude potřeba udelat: 
 - implementace mapového okna - esri api atd
 - napojení databáze s daty
 - jakmile bude mapové okno tak více css rozdělení do sloupce 
 

